In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy


In [3]:
test_features = pd.read_csv('./data/test_features.csv')
train_features = pd.read_csv('./data/train_features.csv')
train_labels = pd.read_csv('./data/train_labels.csv')

In [4]:
train_features.head(5)

,id,filepath,site
0,ZJ000000,train_features/ZJ000000.jpg,S0120
1,ZJ000001,train_features/ZJ000001.jpg,S0069
2,ZJ000002,train_features/ZJ000002.jpg,S0009
3,ZJ000003,train_features/ZJ000003.jpg,S0008
4,ZJ000004,train_features/ZJ000004.jpg,S0036


In [5]:
test_features.head(5)

,id,filepath,site
0,ZJ016488,test_features/ZJ016488.jpg,S0082
1,ZJ016489,test_features/ZJ016489.jpg,S0040
2,ZJ016490,test_features/ZJ016490.jpg,S0040
3,ZJ016491,test_features/ZJ016491.jpg,S0041
4,ZJ016492,test_features/ZJ016492.jpg,S0040


In [6]:
train_labels.head(5)

,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,ZJ000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ZJ000001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,ZJ000002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ZJ000003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,ZJ000004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
def merge_animals(df):
    """_summary_

    Args:
        df (_type_): _description_

    Returns:
        _type_: _description_
    """
    df_copy = df.copy() # copy df
    columns = df_copy.columns # get column names
    df_copy['animal_classification'] = np.where(df_copy.values)[1]+1 # add a numeric value to each column
    df_copy.drop(columns ,axis=1, inplace=True) # drop columns that were just combined
    return df_copy
#                ['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']
# Classification:           1           2        3           4          5        6              7               8

In [5]:
# combine train df's on id
train = pd.merge(left=train_features, right=train_labels, on='id') 

# see function in above cell
train['animal_classification'] = merge_animals(train[['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']]) 
# done in function above ^ might need 
# train.drop(['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent'] ,axis=1, inplace=True) # drop

# rename numeric observations to actual classifications
train['animal_classification'] = train['animal_classification'].map({1:'antelope_duiker', 2:'bird', 3:'blank', 4:'civet_genet', 5:'hog', 6:'leopard', 7:'monkey_prosimian', 8:'rodent'})

# split file path column to get file names
temp = train['filepath'].str.split(pat='/',expand=True)
# rename split columns
temp.rename(columns={0: 'old_folder_location', 1: 'filename'}, inplace=True)

# concat columns and original df
train = pd.concat([train, temp], axis=1).drop(columns=['filepath'],axis=1)


In [6]:
# finding better site number that are more balanced
print(train[train['site'] == 'S0060']['animal_classification'].value_counts())
print(train[train['site'] == 'S0009']['animal_classification'].value_counts())

civet_genet         959
antelope_duiker      59
monkey_prosimian     49
blank                23
rodent               22
hog                  20
Name: animal_classification, dtype: int64
monkey_prosimian    190
hog                 188
bird                155
blank                57
antelope_duiker      33
rodent               25
civet_genet          16
Name: animal_classification, dtype: int64


In [7]:
# make validation set
validation_set = train[(train['site']=='S0009') | (train['site']=='S0043')| (train['site']=='S0059') |(train['site']== 'S0026')] # get validation set for 2 sites
# make training set
train_set = train[~train.isin(validation_set)].dropna() # remove the observations from train that are in the validation set

In [8]:
print(validation_set['site'].value_counts(normalize=True))
print(train_set['site'].value_counts(normalize=True))

S0009    0.402424
S0043    0.269091
S0059    0.265455
S0026    0.063030
Name: site, dtype: float64
S0060    0.076291
S0063    0.037539
S0008    0.036460
S0036    0.030732
S0038    0.028912
           ...   
S0143    0.000202
S0078    0.000135
S0079    0.000135
S0178    0.000135
S0102    0.000067
Name: site, Length: 144, dtype: float64


In [9]:
print(validation_set['site'].value_counts(normalize=True))
print(train_set['site'].value_counts(normalize=True))

S0009    0.402424
S0043    0.269091
S0059    0.265455
S0026    0.063030
Name: site, dtype: float64
S0060    0.076291
S0063    0.037539
S0008    0.036460
S0036    0.030732
S0038    0.028912
           ...   
S0143    0.000202
S0078    0.000135
S0079    0.000135
S0178    0.000135
S0102    0.000067
Name: site, Length: 144, dtype: float64


In [11]:
# image Gen stuff
# ['antelope_duiker',	'bird',	'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian','rodent']
img_gen = ImageDataGenerator()
val_generator = img_gen.flow_from_dataframe(
    validation_set, 
    directory='./data/train_features_img', 
    x_col='filename', 
    y_col='animal_classification', 
    target_size=(256, 256), 
    class_mode='categorical',
    batch_size=64
)
train_generator = img_gen.flow_from_dataframe(
    train_set, 
    directory='./data/train_features_img', 
    x_col='filename', 
    y_col='animal_classification', 
    target_size=(256, 256), 
    class_mode='categorical',
    batch_size=64
)

Found 1650 validated image filenames belonging to 8 classes.
Found 14838 validated image filenames belonging to 8 classes.


In [12]:
def plot_metrics(model_fit):
    metrics = ['accuracy', 'precision', 'recall']
    for i in metrics:
        plt.plot(model_fit.history[i], label='Train')
        plt.plot(model_fit.history[f'val_{i}'], label='Test')
        plt.ylabel(i)
        plt.xlabel('Epochs')
        plt.legend()
        plt.show()

In [13]:
model = Sequential()
# layers
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
# output layer
model.add(Dense(units=8, activation='sigmoid'))

In [14]:
model.compile(loss=categorical_crossentropy, optimizer='adam', metrics=['accuracy', 'Recall', 'Precision'])

In [15]:
history = model.fit(
    train_generator,
    batch_size=64,
    epochs=50,
    validation_data=val_generator
)


Epoch 1/50
232/232 [==============================] - 138s 594ms/step - loss: 440.0434 - accuracy: 0.2241 - recall: 0.4698 - precision: 0.1662 - val_loss: 2.2956 - val_accuracy: 0.0661 - val_recall: 0.2818 - val_precision: 0.0940
Epoch 2/50
232/232 [==============================] - 115s 497ms/step - loss: 1.9693 - accuracy: 0.2382 - recall: 0.6363 - precision: 0.1688 - val_loss: 2.1160 - val_accuracy: 0.0661 - val_recall: 0.6194 - val_precision: 0.1251
Epoch 3/50
232/232 [==============================] - 114s 492ms/step - loss: 1.8760 - accuracy: 0.2524 - recall: 0.7906 - precision: 0.1666 - val_loss: 2.1142 - val_accuracy: 0.0727 - val_recall: 0.6164 - val_precision: 0.1269
Epoch 4/50
232/232 [==============================] - 114s 491ms/step - loss: 1.8179 - accuracy: 0.2760 - recall: 0.8118 - precision: 0.1740 - val_loss: 2.1380 - val_accuracy: 0.0727 - val_recall: 0.6055 - val_precision: 0.1269
Epoch 5/50
232/232 [==============================] - 115s 495ms/step - loss: 1.7592 -

KeyboardInterrupt: 

In [ ]:
plot_metrics(history)